#### Run ROI-level analyses 
    - plot unphased and phased timeseries 
    - plot frequency spectrum
    - reliability metrics
sbatch --time=10:00:00 --cpus-per-task=2 --mem-per-cpu=8GB --account=def-mmur {script}

In [1]:
from pathlib import Path
import itertools

FO_THR = .8
PCORR = "fdrp"
smooth_id = 0
n_iterations = 25
n_batches_expected = 16
time_window = (39, 219)
TR = .3
base_dir = "/scratch/fastfmri"

def get_im_frequencies(f1,f2):
    assert f2 > f1, f"{f2} <= {f1}"
    im_frequencies = {}
    im_frequencies["first_order"] = [f1, f2]
    im_frequencies["second_order"] = [
        round(f2-f1, 10),
        round(f1+f2, 10),
        round(f1*2, 10),
        round(f2*2, 10),
    ]
    im_frequencies["third_order"] = [
        round(2*f1-f2, 10),
        round(2*f2-f1, 10),
    ]

    return im_frequencies

3T/7T Vary

In [2]:
experiment_id = "1_frequency_tagging"
mri_ids = ["3T", "7T"]
# Iterate over these options
desc_ids = ["IMall"]*3
roi_task_ids = ["entrainD", "entrainE", "entrainF"]
task_ids = [None, None, None]
task_frequencies = [[.125, .2], [.15, .2], [.175,.2]]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

for mri_id in mri_ids:
    for sub_id in ["021"]:

        for desc_id, roi_task_id, task_id, _task_frequencies in zip(desc_ids, roi_task_ids, task_ids, task_frequencies):
            for im_order, im_task_frequencies in get_im_frequencies(_task_frequencies[0], _task_frequencies[1]).items():
                for task_frequency in im_task_frequencies:
                    _task_id = roi_task_id
                    if task_id is not None:
                        _task_id = task_id
                    print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{task_frequency}], task: {_task_id} {[task_frequency]}")

                    # Check, and run if it doesn't exist
                    out_dir_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
                    )
                    out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{_task_id}_bootstrapped_data.pkl"
                    if out_file.exists():
                        continue

                    txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {task_frequency} \\
    --task-conditions {_task_id} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in [task_frequency]])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
                    """
                    out_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_task-{_task_id}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
                    )
                    cmd_path = scripts_dir / out_base
                    print(cmd_path)
                    with open(cmd_path, 'w') as f:
                        f.write(txt)

[IMall] sub-021, roi: entrainD [0.125], task: entrainD [0.125]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainD-0.125_task-entrainD_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-021, roi: entrainD [0.2], task: entrainD [0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainD-0.2_task-entrainD_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-021, roi: entrainD [0.075], task: entrainD [0.075]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainD-0.075_task-entrainD_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-021, roi: entrainD [0.325], task: entrainD [0.325]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainD-0.325_ta

In [3]:
experiment_id = "1_frequency_tagging"
mri_ids = ["3T", "7T"]
# Iterate over these options
desc_ids = ["IMall"]*3
roi_task_ids = ["entrainA", "entrainB", "entrainC"]
task_ids = [None, None, None]
task_frequencies = [[.125, .2], [.125, .175], [.125,.15]]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

for mri_id in mri_ids:
    for sub_id in ["020"]:

        for desc_id, roi_task_id, task_id, _task_frequencies in zip(desc_ids, roi_task_ids, task_ids, task_frequencies):
            for im_order, im_task_frequencies in get_im_frequencies(_task_frequencies[0], _task_frequencies[1]).items():
                for task_frequency in im_task_frequencies:
                    _task_id = roi_task_id
                    if task_id is not None:
                        _task_id = task_id
                    print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{task_frequency}], task: {_task_id} {[task_frequency]}")

                    # Check, and run if it doesn't exist
                    out_dir_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
                    )
                    out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{_task_id}_bootstrapped_data.pkl"
                    if out_file.exists():
                        continue

                    txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {task_frequency} \\
    --task-conditions {_task_id} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in [task_frequency]])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
                    """
                    out_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_task-{_task_id}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
                    )
                    cmd_path = scripts_dir / out_base
                    print(cmd_path)
                    with open(cmd_path, 'w') as f:
                        f.write(txt)

[IMall] sub-020, roi: entrainA [0.125], task: entrainA [0.125]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainA-0.125_task-entrainA_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-020, roi: entrainA [0.2], task: entrainA [0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainA-0.2_task-entrainA_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-020, roi: entrainA [0.075], task: entrainA [0.075]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainA-0.075_task-entrainA_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-020, roi: entrainA [0.325], task: entrainA [0.325]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrainA-0.325_ta

3T/7T Normal

In [4]:
mri_ids = ["3T", "7T"]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

for mri_id in mri_ids:
    if mri_id == "3T":
        sub_ids = ["000", "002", "003", "004", "005", "006", "007", "008", "009"]
        experiment_id = "1_frequency_tagging"
        desc_ids = ["IMall"]*4
        roi_task_ids = ["entrain"]*2 + ["control"]*2
        task_ids = ["entrain", "control"]*2
        task_frequencies = [[.125, .2], [.125,.2]]*2
    elif mri_id == "7T":
        sub_ids = ["Pilot001", "Pilot009", "Pilot010", "Pilot011"]
        experiment_id = "1_attention"
        desc_ids = ["IMall"]*3
        roi_task_ids = ["AttendAway"]
        task_ids = [None]
        task_frequencies = [[.125, .2]]

    for sub_id in sub_ids:
        for desc_id, roi_task_id, task_id, _task_frequencies in zip(desc_ids, roi_task_ids, task_ids, task_frequencies):
            for im_order, im_task_frequencies in get_im_frequencies(_task_frequencies[0], _task_frequencies[1]).items():
                for task_frequency in im_task_frequencies:
                    _task_id = roi_task_id
                    if task_id is not None:
                        _task_id = task_id

                    print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{task_frequency}], task: {_task_id} {[task_frequency]}")

                    # Check, and run if it doesn't exist
                    out_dir_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
                    )
                    out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{_task_id}_bootstrapped_data.pkl"
                    if out_file.exists():
                        continue

                    txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {task_frequency} \\
    --task-conditions {_task_id} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in [task_frequency]])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
                    """
                    out_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_task-{_task_id}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
                    )
                    cmd_path = scripts_dir / out_base
                    print(cmd_path)
                    with open(cmd_path, 'w') as f:
                        f.write(txt)

[IMall] sub-000, roi: entrain [0.125], task: entrain [0.125]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrain-0.125_task-entrain_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-000, roi: entrain [0.2], task: entrain [0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrain-0.2_task-entrain_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-000, roi: entrain [0.075], task: entrain [0.075]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrain-0.075_task-entrain_pval-fdrp_fo-0.8_bootstrap.sh
[IMall] sub-000, roi: entrain [0.325], task: entrain [0.325]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-IMall_roi-entrain-0.325_task-entrain_pval

### Not part of frequency tagging study
- Attention

In [ ]:
mri_ids = ["7T"]
smooth_id = 0
n_iterations = 100
n_batches_expected = 2
time_window = (39, 219)
TR = .25
base_dir = "/scratch/fastfmri"

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)

for mri_id in mri_ids:
    if mri_id == "7T":
        sub_ids = ["010", "011", "012", "013", "014", "015", "016"]
        experiment_id = "1_attention"
        desc_ids = ["IMall"]*4
        roi_task_ids = ["AttendAway", "AttendInF1", "AttendInF2", "AttendInF1F2"]
        task_ids = [None, None, None, None]
        task_frequencies = [[.125, .2], [.125, .2], [.125, .2], [.125, .2]]

    for sub_id in sub_ids:
        for desc_id, roi_task_id, task_id, _task_frequencies in zip(desc_ids, roi_task_ids, task_ids, task_frequencies):
            for im_order, im_task_frequencies in get_im_frequencies(_task_frequencies[0], _task_frequencies[1]).items():
                for task_frequency in im_task_frequencies:
                    _task_id = roi_task_id
                    if task_id is not None:
                        _task_id = task_id

                    print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{task_frequency}], task: {_task_id} {[task_frequency]}")

                    # Check, and run if it doesn't exist
                    out_dir_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
                    )
                    out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{_task_id}_bootstrapped_data.pkl"
                    if out_file.exists():
                        continue

                    txt = f"""#!/bin/bash
module load apptainer/1.1

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {task_frequency} \\
    --task-conditions {_task_id} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in [task_frequency]])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
                    """
                    out_base = (
                        f"experiment-{experiment_id}"
                        f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
                        f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{task_frequency}_task-{_task_id}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
                    )
                    cmd_path = scripts_dir / out_base
                    print(cmd_path)
                    with open(cmd_path, 'w') as f:
                        f.write(txt)